# Set up OpenAI and start with a system prompt

In [152]:
import json
import os

from openai import OpenAI

In [153]:
EXAMPLES_DIR = 'examples'
os.makedirs(EXAMPLES_DIR, exist_ok=True)

In [154]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=open('../key.txt', 'r').read().strip('\n'),
)

In [155]:
system_prompt = """
# Instructions:
You are an autocomplete system for Python. Generate or complete Python code based on the user's description or the provided code snippet.

The user may
1. Describe code, either directly or in a Python comment. In this case, you should implement the described code.
2. Write incomplete code. In this case, you should complete the code from the last cursor position.
3. If the last character is a newline or tab, come up with one or more possible next lines of code.


## Example 1:

[INPUT]
# Function that adds two numbers
def
[/INPUT]

[ANSWER]
```python
 add_numbers(a, b):
    return a + b
```
[/ANSWER]


## Example 2:

[INPUT]
```python
def factorial(n):
    # Computes the fact
```
[/INPUT]

[ANSWER]
```python
    # Computes the factorial of a number
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
```
[/ANSWER]


## Example 3:

[INPUT]
I need a function that checks if a number is prime
[/INPUT]

[ANSWER]
```python
def is_prime(number):
    if number > 1:
        for i in range(2, int(number**0.5)+1):
            if (number % i) == 0:
                return False
        return True
    else:
        return False
```
[/ANSWER]


# Rules:
- Your entire response must be valid Python code.
- Avoid evasive or incomplete answers.
- Any comments or notes must be made in Python code block, i.e., ```python ... ```.
- You must not add any additional comments or notes outside of the ```python ... ``` blocks. Your entire response must only consist of one Python code block.
- You must end your response after ending the Python code block with "```".
"""

In [156]:
def add_message(messages: list[str], role: str, message: str | None = None) -> list[str]:
    # If the user is the assistant, generate a response
    if role == "assistant" and message is None:
        chat_completion = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo")
        message = chat_completion.choices[0].message.content

    # Add the message to the messages list
    messages.append({
        "role": role,
        "content": message,
    })

    return messages

# Generate a response to an example prompt

In [157]:
user_input = '''def get_document_info_page(url):
    """
    Get the redirected url from a given url

    Example:
    https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:31983H0230&qid=1698155004044&rid=460 -> https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:31983H0230
    """

    '''

In [158]:
messages = []

In [159]:
messages = add_message(messages, "system", system_prompt)
messages = add_message(messages, "user", user_input)

In [160]:
messages = add_message(messages, "assistant")

# Examine the output

In [161]:
print(messages[-1]['content'])

```python
from urllib.parse import urlparse

def get_document_info_page(url):
    """
    Get the redirected url from a given url

    Example:
    https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:31983H0230&qid=1698155004044&rid=460 -> https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:31983H0230
    """

    parsed_url = urlparse(url)
    path = parsed_url.path
    query = parsed_url.query

    # Remove the unnecessary query parameters
    query = query.replace('qid', '').replace('rid', '')

    # Reconstruct the new url
    new_url = f"{parsed_url.scheme}://{parsed_url.netloc}{path}?{query}"
    
    return new_url
```

```


# Comment & Title

In [163]:
comment = """
- results in invalid url (e.g. 'https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:31983H0230&=1698155004044&=460') because it only removes the parameters but not their values


Ground Truth:
```python
url = url.replace('AUTO', 'EN/ALL')
    return url[:url.find('&')]
```
"""

title = "Truncate URL gone wrong"

In [164]:
# Remove any previous comments
messages = [message for message in messages if message['role'] != 'comment']

# Add the comment to the messages list
messages.append({
    "role": "comment",
    "content": comment,
})

In [165]:
# Save the messages to a file
if not os.path.exists(os.path.join(EXAMPLES_DIR, f"{title}.json")):
    with open(os.path.join(EXAMPLES_DIR, f"{title}.json"), "w") as f:
        json.dump(messages, f, indent=4)
else:
    print(f"File {title}.json already exists. Skipping.")